In [1]:
# Decision Tree (CART, ID3, C4.5, etc.)

In [2]:
# Random Forest (RF, 4.5 “bridge” between DT and boosting)

In [4]:
# Gradient Boosted Decision Trees (GBDT)
# Good link: https://www.showmeai.tech/article-detail/193

In [5]:
# XGBoost / LightGBM / CatBoost
# https://zhuanlan.zhihu.com/p/142413825

In [6]:
# Other Notable Tree Variants

# 🌳 Decision Tree Family: ID3, C4.5, C5.0, CART

---

## 🔢 Entropy & Information Gain (core math first)

Let dataset $\mathcal{D}$, target $Y$, candidate feature $X$.

**Entropy of target:**
$$
H(Y) = - \sum_{c} p(c) \log p(c)
$$

**Conditional entropy (split on $X$):**
- If $X$ is categorical:
$$
H(Y|X) = \sum_{v \in \mathcal{V}} p(X=v) \; H(Y | X=v)
$$

- If $X$ is continuous with threshold $t$:
$$
H(Y|X \leq t) = \frac{|\mathcal{D}_{\leq t}|}{|\mathcal{D}|} H(Y | X \leq t) + \frac{|\mathcal{D}_{>t}|}{|\mathcal{D}|} H(Y | X > t)
$$

**Information Gain:**
$$
IG(Y,X) = H(Y) - H(Y|X)
$$

**Split Information (intrinsic info):**
$$
SI(X) = - \sum_{v \in \mathcal{V}} p(X=v) \log p(X=v)
$$

**Gain Ratio (used by C4.5):**
$$
GR(Y,X) = \frac{IG(Y,X)}{SI(X)}
$$

---

## 📜 ID3 (Quinlan, 1980s)

- **Split criterion:** Information Gain.
- **Features:** Categorical only (no native continuous).
- **Splits:** Multiway (branch for each category).
- **Pruning:** None/minimal → tends to overfit.
- **Missing values:** Not handled.

---

## 📜 C4.5 (1993, Quinlan)

- **Split criterion:** Gain Ratio (fixes ID3’s bias toward high-cardinality features).
- **Continuous variables:** Handled natively (binary threshold splits).
- **Missing values:** Handled via fractional instance weighting.
- **Splits:** Multiway (categorical), binary (continuous).
- **Pruning:** Error-based pruning (see below).

### 🔍 Error-based pruning explained
1. Grow the full tree (overfitted).
2. For each subtree, estimate **expected error** using a binomial confidence interval:
   - If replacing the subtree with a single leaf has **similar expected error**, prune it.
3. Intuition: keep only splits that reduce error *confidently*.

This avoids overfitting while not being as aggressive as CART’s pruning.

---

## 📜 C5.0 (Commercial successor)

At first glance it looks like “just a faster C4.5”, but it includes real upgrades:

- **Memory/computation:**
  - C4.5: builds full tree in memory.
  - C5.0: compressed data structures (bitmaps, caching) → much lower memory, faster splits.

- **Boosting:** Built-in AdaBoost-like option.
- **Winnowing:** Drops irrelevant features automatically.
- **Cost-sensitive:** Allows weighting of misclassification types.
- **Rule sets:** Can convert trees into compact, interpretable rules.
- **Overall:** Same logical core as C4.5, but industrial-strength for large, noisy datasets.

---

## 📜 CART (Breiman et al., 1984)

- **Split criteria:**
  - Classification: Gini impurity (or entropy).
    $$
    Gini(Y) = 1 - \sum_c p(c)^2
    $$
  - Regression: MSE (variance reduction).
- **Splits:** Always **binary**, even for categorical features.
- **Regression support:** Native (variance).
- **Missing values:** Handled via surrogate splits.
- **Pruning:** Cost–complexity pruning (details below).

---

## 📈 CART for Regression

CART isn’t limited to classification — it was designed to also support **regression tasks**.
This makes it more general-purpose than ID3/C4.5, which are classification-only.

### 🔹 Impurity Measure for Regression
Instead of entropy or Gini, CART uses **variance** (equivalently MSE).

At a node $t$ with target values $\{y_i\}_{i=1}^N$:

$$
Var(t) = \frac{1}{N} \sum_{i=1}^N (y_i - \bar{y})^2, \quad \bar{y} = \frac{1}{N}\sum_{i=1}^N y_i
$$

### 🔹 Split Selection
When splitting a node into left/right children:

$$
\Delta Var = Var(parent) - \Big( \frac{N_L}{N} Var(L) + \frac{N_R}{N} Var(R) \Big)
$$

- $N_L$, $N_R$: number of samples in left/right child.
- Choose the split that **maximizes variance reduction** (just like maximizing Gini reduction in classification).

### 🔹 Predictions at Leaves
- **Classification:** majority class in leaf.
- **Regression:** average of the target values in leaf:
  $$
  \hat{y}_{leaf} = \frac{1}{N_{leaf}} \sum_{i \in leaf} y_i
  $$

### 🔹 Why This Matters
- ID3 / C4.5 use entropy-based criteria, which only apply to discrete class distributions.
- CART defined splitting more generally:
  - Classification → Gini/entropy.
  - Regression → Variance/MSE.
- This dual design made CART the **foundation for Random Forests and Gradient Boosting**, which often need regression trees internally (e.g., GBDT fits residuals with regression splits).

---

## ✂️ CART’s Cost–Complexity Pruning

Define cost:
$$
R_\alpha(T) = R(T) + \alpha |T|
$$
- $R(T)$: empirical error (misclass or MSE).
- $|T|$: number of leaves.
- $\alpha$: complexity penalty.

### Weakest link pruning (back-merging)
1. Start with the full tree.
2. For each internal node $t$, compute:
   $$
   \alpha(t) = \frac{R(t) - R(T_t)}{|T_t|-1}
   $$
   where $T_t$ is the subtree rooted at $t$.
   - Intuition: error increase per leaf removed.
3. Find the node with smallest $\alpha(t)$ (“weakest link”).
4. Collapse its subtree into a leaf.
5. Repeat until only the root remains.

This produces a **sequence of nested trees**:
$$
T_0 \supset T_1 \supset T_2 \supset \dots \supset T_m
$$

Finally, choose the optimal subtree via **cross-validation**.

---

## ❓ Clarification Points (Q&A merged into notes)

### Q1: Difference between C4.5 and C5.0?
- C5.0 isn’t just “faster.” It adds boosting, feature winnowing, cost-sensitive learning, memory compression, and rule extraction. Much more production-ready.

### Q2: C4.5 pruning — what does "error-based pruning after full tree" mean?
- It means grow first, then prune back subtrees where error reduction isn’t statistically significant, using confidence intervals.

### Q3: Details for CART pruning?
- CART uses cost–complexity pruning with weakest link merging. See formula for $\alpha(t)$ above. Produces full pruning path down to a single leaf.

### Q4: Why CART is better for modern ensembles?
- Ensembles (Random Forest, GBDT, XGBoost) require:
  - **Binary splits** (CART enforces this).
  - **Regression support** (variance reduction


In [8]:
from collections import Counter
import math
def learn_decision_tree(examples: list[dict], attributes: list[str], target_attr: str) -> dict:
  # Your code here
  # Step 0 If there's only one feature, do majority count
  if len(attributes) == 1:
    a = attributes[0]
    attr_to_target = {}
    for e in examples:
        if e[a] not in attr_to_target:
            attr_to_target[e[a]] = []
            attr_to_target[e[a]].append(e[target_attr])
    return {a: {k: majority(v) for k,v in attr_to_target.items()}}
  # Step 1 Build y_vals, {attribute: {attribute_val: [y_vals]}}
  y_vals = []
  attr_to_y_vals = {}
  for e in examples:
      y_vals.append(e[target_attr])
      for a in attributes:
          if a not in attr_to_y_vals:
              attr_to_y_vals[a] = {}
          if e[a] not in attr_to_y_vals[a]: attr_to_y_vals[a][e[a]] = []
          attr_to_y_vals[a][e[a]].append(e[target_attr])
  # Step 2 Check IG # Since we only want to compare IG, and they will all be h(y) - h(y|x), so we jsut pick the one with smallest h(y|x)
  attr_to_ig = {}
  for a in attributes:
      attr_to_ig[a] = h_x(attr_to_y_vals[a])

  # Step 3 Choose the best attributes
  lowest_key = min(attr_to_ig.items(), key=lambda kv: (kv[1], kv[0]))[0]
  new_attributes = attributes.copy()
  new_attributes.remove(lowest_key)
  dt = {lowest_key: {}}
  for k, v in attr_to_y_vals[lowest_key].items():
      if len(set(v)) == 1:
          dt[lowest_key][k] = v[0]
      else:
          dt[lowest_key][k] = learn_decision_tree([e for e in examples if e[lowest_key] == k], new_attributes, target_attr)
  # a. if y pure -> val # b. if y not pure -> recurssion
  return dt
def majority(y_vals: list[str]):
    counts = Counter(y_vals)
    majority_label, _ = counts.most_common(1)[0]
    return majority_label

def h(feature: list[str]):
    count = {}
    for f in feature:
      if f in count:
          count[f] += 1
      else:
          count[f] = 1
    for k in count.keys():
        count[k] /= len(feature)
    h_val = 0
    for v in count.values():
        h_val -= v*math.log(v)
    return h_val

def h_x(feature: dict[str, list[str]]):
    h_sum = 0
    items = 0
    for x, y_vals in feature.items():
        h_sum += h(y_vals)*len(y_vals)
        items += len(y_vals)
    return h_sum/items

print(learn_decision_tree([ {'Outlook': 'Sunny', 'Wind': 'Weak', 'PlayTennis': 'No'}, {'Outlook': 'Overcast', 'Wind': 'Strong', 'PlayTennis': 'Yes'}, {'Outlook': 'Rain', 'Wind': 'Weak', 'PlayTennis': 'Yes'}, {'Outlook': 'Sunny', 'Wind': 'Strong', 'PlayTennis': 'No'}, {'Outlook': 'Sunny', 'Wind': 'Weak', 'PlayTennis': 'Yes'}, {'Outlook': 'Overcast', 'Wind': 'Weak', 'PlayTennis': 'Yes'}, {'Outlook': 'Rain', 'Wind': 'Strong', 'PlayTennis': 'No'}, {'Outlook': 'Rain', 'Wind': 'Weak', 'PlayTennis': 'Yes'} ], ['Outlook', 'Wind'], 'PlayTennis'))

{'Outlook': {'Sunny': {'Wind': {'Weak': 'No', 'Strong': 'No'}}, 'Overcast': 'Yes', 'Rain': {'Wind': {'Weak': 'Yes', 'Strong': 'No'}}}}


In [9]:
# https://www.deep-ml.com/problems/138


# Gradient Boosted Decision Trees (GBDT) — practical, intuition-first notes

> These are copy-paste friendly notebook notes (markdown cells). Math is kept light and only where it sharpens intuition.

---

## What GBDT is (in one breath)

- We build a **prediction function** \(F(x)\) as a **sum of small trees**:
  $$
  F_M(x) = F_0(x) + \nu \sum_{m=1}^{M} \gamma_m\, T_m(x)
  $$
  where each small regression tree \(T_m\) nudges \(F\) a little, \(\nu\) is a **learning rate**, and \(\gamma_m\) is a step size.

- For **binary classification**, we usually let \(F(x)\) represent **log-odds** and then map it to probability with the sigmoid:
  $$
  p(x) = \sigma(F(x)) = \frac{1}{1+e^{-F(x)}}
  $$

- Training is **stage-wise**: at each round we add one more tree that reduces the chosen loss on the current model’s mistakes.

---

## Why “boost the gradient” (and not the residual) — using logistic loss

### The short answer
- “Residuals” are only the right error notion for **squared error**. For other losses, “residual” doesn’t align with the loss surface.
- The **gradient** of the loss tells you the direction of steepest improvement **for the loss you actually care about**. So we fit a weak learner to the **negative gradient** and add it to the model.

### The binary logistic case (intuition, not derivation)
- Logistic loss for a label \(y\in\{0,1\}\) and predicted probability \(p=\sigma(F)\) is:
  $$
  \ell(y,p) = -\big[y\log p + (1-y)\log(1-p)\big]
  $$
- If you ask “what small change \(\delta\) in the **score** \(F\) would most reduce \(\ell\) right now?”, calculus answers: **move in the negative gradient** of \(\ell\) w.r.t. \(F\).
- For logistic loss, that negative gradient at each sample becomes:
  $$
  -\frac{\partial \ell}{\partial F} = y - p
  $$
  which you can read as **“desired probability minus current probability.”**

- Key intuition: in classification, your target is **not** to match labels directly with the raw score \(F\); it’s to make **probabilities** correct after the **link** \(p=\sigma(F)\). The gradient **bakes in the link** and the loss shape automatically.

- Using “residual = label − prediction” only makes sense when the prediction is on the same scale as the target **and** the loss is squared error. With logistic, the right “residual-like” signal is exactly the **negative gradient** \(y-p\) in **score space**.

### Takeaways
- Boosting = **functional gradient descent**: each tree approximates the negative gradient field over \(x\).
- This generalizes to **any differentiable loss** (ranking, Poisson, quantile, etc.). Residuals don’t.

---

## Why use CART regression trees as weak learners?

### Practical reasons
- **Piecewise-constant tweaks**: regression trees predict a constant per leaf. Perfect for **local adjustments** to \(F(x)\) in regions where the gradient says “push up” or “push down.”
- **Captures interactions**: Even shallow trees model **feature interactions**. Stacking many gives strong non-linearity.
- **Scale-free, low prep**: Minimal preprocessing needed, robust to feature scaling and missingness.
- **Fast fitting**: Many small trees are computationally efficient with histogram/approximate split finding.
- **Simple regularization knobs**: Depth, min samples per leaf, L2 on leaf values, subsampling.

### Conceptual fit
- At each round, we have a target “signal” (the negative gradient per sample). A regression tree is a **universal, cheap function approximator** that fits that signal piecewise.
- With second-order updates (XGBoost/LightGBM), leaf values are set by Newton updates (using gradients + Hessians), and CART leaves (constants) make that closed-form.

---

## Anatomy of one boosting round (binary logistic)

1. Current model: \(F_{m-1}(x)\) → probabilities \(p=\sigma(F_{m-1}(x))\).
2. Compute pseudo-residuals: \(r_i = y_i - p_i\).
3. Fit a small regression tree \(T_m(x)\) to \(\{(x_i, r_i)\}\).
4. Compute step sizes per leaf (line search or Newton step).
5. Update model:
   $$
   F_m(x) = F_{m-1}(x) + \nu \cdot \gamma_m T_m(x)
   $$

> Each tree is a local “probability correction map” saying: *in this region, we’re under-predicting positives by ~0.03; bump the score a bit.*

---

## The role of learning rate \(\nu\)

- Small \(\nu\) = cautious steps, many trees → smoother path, better generalization.
- Large \(\nu\) = fewer trees, but high risk of overfit.
- Typical: \(\nu \in [0.01, 0.1]\) with hundreds–thousands of trees.

---

## Regularization that matters

- **Tree depth**: shallow trees = simpler corrections = less variance.
- **Min samples/weight per leaf**: avoids fitting noise.
- **L2 on leaf values**: shrinks extreme corrections.
- **Row/col subsampling**: adds diversity, stabilizes trees.
- **Early stopping**: prevents fitting noise.

---

## Initialization & class imbalance

- Start with \(F_0\) = log-odds of base rate. Avoids wasting trees rediscovering prior.
- For imbalance:
  - Use **class weights** or **scale_pos_weight**.
  - Tune threshold post-training for your metric.
  - Validate with PR-AUC / recall when imbalance is severe.

---

## GBDT vs linear models

- **GBDT wins**: strong feature interactions, thresholds, minimal preprocessing.
- **Linear wins**: very high-dim sparse data, smooth extrapolation, calibrated probabilities.

---

## CART vs alternatives

- **CART regression trees**: workhorse, piecewise constant.
- **Oblivious/symmetric trees (CatBoost)**: same split at each depth, efficient with categoricals.
- **Linear leaves**: add local linear trends, costlier.

---

## First-order vs second-order

- First-order: gradients only, stable.
- Second-order: gradients + Hessians, faster convergence. Hessian naturally downweights extreme/saturated points.

---

## Practical tuning recipe

1. Shallow trees (depth 3–6).
2. \(\nu \sim 0.05–0.1\).
3. Subsample rows/columns.
4. Use early stopping.
5. If overfitting → smaller \(\nu\), shallower trees, stronger L2, bigger min leaf, more subsampling.
6. Calibrate probabilities if needed.

---

## Interpretability

- **Feature importance**: what features matter.
- **Partial dependence / ICE**: how features shift score.
- **SHAP**: per-feature contributions.

---

## Missing values & categoricals

- Trees can branch on NaNs → no imputation needed.
- CatBoost/LightGBM handle categoricals natively; otherwise use careful encoding.

---

## Pitfalls

- Too deep + high LR = overfit.
- No early stopping = wasted trees.
- Ignoring imbalance = misleading metrics.
- Target leakage with encodings = catastrophic overfit.
- Tiny validation = unstable early stopping.

---

## Boosting vs bagging

- **Bagging (Random Forests)**: deep independent trees, reduce variance by averaging.
- **Boosting (GBDT)**: shallow sequential trees, reduce bias by correcting gradients.

---

## Checklist for future self

- ✅ Work in **score space**.
- ✅ Use negative gradients, not residuals.
- ✅ Prefer small trees + shrinkage.
- ✅ Always early stop.
- ✅ Handle imbalance explicitly.
- ✅ Choose library wisely (XGB, LGBM, CatBoost).
- ✅ Interpret with SHAP.

---

## Minimal recap

- Predict score \(F(x)\).
- Probability = \(\sigma(F)\).
- Negative gradient = error signal.
- Fit tree to it, add it in.
- Use shrinkage + regularization.
- That’s GBDT.


# GBDT — Interview Q&A Cheatsheet

---

## Q: What is boosting?
- **Answer:** Boosting is an ensemble technique where we build models sequentially, each new model focusing on correcting the mistakes of the previous ones.
- In GBDT, the correction is guided by the **negative gradient of the loss**, so each tree learns where the current model is wrong and nudges predictions in the right direction.
- Compared to bagging (like Random Forests), boosting reduces **bias** more than variance.

---

## Q: How is GBDT different from Random Forests?
- **Random Forests (bagging):**
  - Train many **deep trees independently** on bootstrapped samples.
  - Use **Gini/entropy** as split criteria (classification).
  - Reduce **variance** via averaging.
- **GBDT (boosting):**
  - Train many **shallow regression trees sequentially**.
  - Each tree fits the **gradient of the loss**.
  - Reduce **bias** by stage-wise functional gradient descent.
- **Key difference:** RF = “majority vote of independent trees,” GBDT = “sum of sequential corrective trees.”

---

## Q: Why do we fit gradients instead of residuals?
- Residuals = correct signal only for squared error (MSE).
- For other losses (e.g. logistic), residuals don’t align with the true loss surface.
- The **gradient of the loss w.r.t. prediction** is the right direction to reduce *any* differentiable loss.
- Example (logistic loss): gradient = \(y - p\), i.e. desired probability minus predicted probability.
- So fitting gradients = aligning each tree with the objective you care about.

---

## Q: How does logistic loss work with GBDT?
- For binary classification:
  - Model outputs a **score** \(F(x)\) (log-odds).
  - Probability = \(\sigma(F(x)) = 1 / (1 + e^{-F(x)})\).
  - Logistic loss: \(-[y\log p + (1-y)\log(1-p)]\).
- The gradient is \(g = p - y\).
- The Hessian is \(h = p(1-p)\).
- Each tree is fit to these gradients/Hessians, providing score corrections that shift probabilities closer to labels.
- Interpretation: trees are not predicting classes directly, they’re **probability corrections** in log-odds space.

---

## Q: How do we prevent overfitting in GBDT?
- **Learning rate (shrinkage):** smaller steps, more trees → smoother fit.
- **Tree depth / num leaves:** shallow trees capture only simple corrections.
- **Min samples/weight per leaf:** avoids splits on noise.
- **Regularization (λ, γ):** penalize large leaf values or unnecessary splits.
- **Subsampling (rows/columns):** adds randomness, reduces correlation between trees.
- **Early stopping:** stop adding trees once validation loss stops improving.
- **Feature engineering & leakage control:** ensure features don’t encode target info.

---

# Quick Tips for Interview
- Emphasize **functional gradient descent** as the key principle of GBDT.
- Always contrast with **Random Forests** (independent bagging vs sequential boosting).
- Show awareness of **Hessians**: “XGBoost/LightGBM use second-order updates for stability.”
- Overfitting controls = learning rate, shallow trees, early stopping.


In [11]:
# Paste into a Jupyter *code* cell and run to render the notes as Markdown
from IPython.display import Markdown, display

md = r"""
# GBDT/XGBoost Split Scoring via Taylor Expansion — One-Cell Notes

## Why Taylor expansion shows up
We add a new tree \(f_t\) to the current ensemble score \(F^{(t-1)}(x)\). The round-\(t\) objective is
$$
\mathcal L^{(t)}=\sum_{i=1}^n \ell\!\big(y_i,\;F^{(t-1)}(x_i)+f_t(x_i)\big)+\Omega(f_t).
$$
Approximate the change in loss with a **second-order Taylor expansion** around \(F^{(t-1)}(x_i)\):
$$
\ell\!\big(y_i,\,F^{(t-1)}(x_i)+f_t(x_i)\big)\;\approx\;\ell\!\big(y_i,\,\hat y_i^{(t-1)}\big)\;+\;g_i\,f_t(x_i)\;+\;\tfrac12\,h_i\,f_t(x_i)^2,
$$
where
$$
g_i=\frac{\partial \ell}{\partial F}\Big|_{F=F^{(t-1)}(x_i)},\qquad
h_i=\frac{\partial^2 \ell}{\partial F^2}\Big|_{F=F^{(t-1)}(x_i)}.
$$

A tree predicts a **constant** \(w_j\) per leaf \(j\). If leaf \(j\) receives samples \(I_j\), its quadratic objective with L2 regularization \(\lambda\) is
$$
\sum_{i\in I_j}\!\Big(g_i w_j+\tfrac12 h_i w_j^2\Big)\;+\;\tfrac{\lambda}{2}w_j^2,
$$
minimized by the **closed-form leaf value**
$$
w_j^\*=-\frac{\sum_{i\in I_j} g_i}{\sum_{i\in I_j} h_i+\lambda}=-\frac{G_j}{H_j+\lambda}.
$$

**Key idea:** The Taylor surrogate gives (1) closed-form leaf predictions and (2) a principled **split gain** used to pick the best (feature, threshold).

---

## How the split (feature + threshold) is chosen
At a node with aggregates \(G=\sum g_i,\;H=\sum h_i\), consider splitting into left/right children with \((G_L,H_L)\) and \((G_R,H_R)\). The **gain** (loss reduction) is
$$
\text{Gain}\;=\;\frac{1}{2}\!\left(\frac{G_L^2}{H_L+\lambda}+\frac{G_R^2}{H_R+\lambda}-\frac{G^2}{H+\lambda}\right)-\gamma.
$$
- Evaluate this **for each candidate threshold of each feature** (exact or histogram/quantile binned).
- Pick the split with **largest positive** Gain; if the best Gain \(\le 0\), **don’t split** (pruned by \(\gamma\)).

This is why modern GBDT libraries optimize **loss reduction via gradients/Hessians**, not Gini/entropy (those are for standalone classification trees/Random Forests).

---

## Symbol glossary (render-safe math)

**Feature vector**
$$x_i \in \mathbb{R}^d$$

**Target**
$$y_i \in \{0,1\}\ \text{or}\ \mathbb{R}$$

**Ensemble score after \(t\) rounds**
$$F^{(t)}(x) \in \mathbb{R}$$

**Prediction (e.g., logistic)**
$$\hat y_i^{(t)}=\sigma\!\big(F^{(t)}(x_i)\big),\quad \sigma(z)=\frac{1}{1+e^{-z}}$$

**Per-sample loss**
$$\ell(y_i,\hat y_i)$$

**Gradient (at current score \(F^{(t-1)}(x_i)\))**
$$g_i=\frac{\partial \ell(y_i,\hat y_i)}{\partial F}\Big|_{F=F^{(t-1)}(x_i)}$$

**Hessian (second derivative at current score)**
$$h_i=\frac{\partial^2 \ell(y_i,\hat y_i)}{\partial F^2}\Big|_{F=F^{(t-1)}(x_i)}$$

**New tree at round \(t\)**
$$f_t(x)$$

**Leaf index set**
$$I_j=\{\,i:\ x_i\ \text{falls into leaf } j\,\}$$

**Leaf value (score the tree outputs)**
$$w_j \in \mathbb{R}$$

**Aggregated gradient/Hessian in leaf \(j\)**
$$G_j=\sum_{i\in I_j} g_i,\qquad H_j=\sum_{i\in I_j} h_i$$

**Optimal leaf value (with L2)**
$$w_j^\*=-\frac{G_j}{H_j+\lambda}$$

**Regularization (XGBoost-style complexity)**
$$\Omega(f)=\gamma T+\frac{\lambda}{2}\sum_{j=1}^T w_j^2\quad\text{(optionally }+\ \alpha\,\lVert w\rVert_{1}\text{)}$$

**Split gain (parent \(G,H\) into left/right \(G_L,H_L\) and \(G_R,H_R\))**
$$
\text{Gain}
=
\frac{1}{2}\!\left(
\frac{G_L^2}{H_L+\lambda}
+
\frac{G_R^2}{H_R+\lambda}
-
\frac{G^2}{H+\lambda}
\right)-\gamma
$$

**Learning rate / ensemble update**
$$F^{(t)}(x)=F^{(t-1)}(x)+\eta\, f_t(x),\qquad \eta\in(0,1]$$

**Binary logistic specific derivatives**
$$g_i=\hat y_i^{(t-1)}-y_i,\qquad h_i=\hat y_i^{(t-1)}\big(1-\hat y_i^{(t-1)}\big)$$

---

## Logistic loss example (binary classification)
Let \(F\) be the score and \(\hat y=\sigma(F)=1/(1+e^{-F})\). With
$$
\ell(y,\hat y)=-\big(y\log \hat y+(1-y)\log(1-\hat y)\big),
$$
we have the derivatives
$$
g=\hat y-y,\qquad h=\hat y(1-\hat y).
$$
Plug these \(g,h\) into \(w^\*=-G/(H+\lambda)\) and the **Gain** formula above.
**Why keep \(h\)?** It yields a **Newton step in function space**, improving step size and split scoring vs. first-order residual fitting—especially important for logistic loss where gradients can saturate.

---

## Practical notes & gotchas
- **Not Gini/entropy:** Boosted trees pick splits by **loss reduction** from the Taylor surrogate, not impurity.
- **Thresholds:** exact (all unique values) vs. **histogram/quantile** binned (default in XGBoost/LightGBM/sklearn-hist).
- **Regularization interacts with splitting:** larger \(\lambda\), \(\gamma\), and `min_child_weight` (minimum child Hessian sum) suppress weak/noisy splits.
- **Missing values (NaN):** per split, learn a **default direction** (left/right) that maximizes Gain—no explicit imputation needed.
- **Early stopping + shrinkage:** low \(\eta\) with many trees and validation-based early stopping is a strong, reliable regularizer.

---

## Interview-style recap (one-liners)
- **Why Taylor expansion?** Quadratic surrogate \(\Rightarrow\) closed-form leaf values + principled split gains.
- **Why Hessians?** Curvature → Newton-like updates; faster, stabler convergence than residual-only fits.
- **How is the split chosen?** Evaluate Gain over (feature, threshold); take the **largest positive**; else stop splitting.
"""

display(Markdown(md))



# GBDT/XGBoost Split Scoring via Taylor Expansion — One-Cell Notes

## Why Taylor expansion shows up
We add a new tree \(f_t\) to the current ensemble score \(F^{(t-1)}(x)\). The round-\(t\) objective is
$$
\mathcal L^{(t)}=\sum_{i=1}^n \ell\!\big(y_i,\;F^{(t-1)}(x_i)+f_t(x_i)\big)+\Omega(f_t).
$$
Approximate the change in loss with a **second-order Taylor expansion** around \(F^{(t-1)}(x_i)\):
$$
\ell\!\big(y_i,\,F^{(t-1)}(x_i)+f_t(x_i)\big)\;\approx\;\ell\!\big(y_i,\,\hat y_i^{(t-1)}\big)\;+\;g_i\,f_t(x_i)\;+\;\tfrac12\,h_i\,f_t(x_i)^2,
$$
where
$$
g_i=\frac{\partial \ell}{\partial F}\Big|_{F=F^{(t-1)}(x_i)},\qquad
h_i=\frac{\partial^2 \ell}{\partial F^2}\Big|_{F=F^{(t-1)}(x_i)}.
$$

A tree predicts a **constant** \(w_j\) per leaf \(j\). If leaf \(j\) receives samples \(I_j\), its quadratic objective with L2 regularization \(\lambda\) is
$$
\sum_{i\in I_j}\!\Big(g_i w_j+\tfrac12 h_i w_j^2\Big)\;+\;\tfrac{\lambda}{2}w_j^2,
$$
minimized by the **closed-form leaf value**
$$
w_j^\*=-\frac{\sum_{i\in I_j} g_i}{\sum_{i\in I_j} h_i+\lambda}=-\frac{G_j}{H_j+\lambda}.
$$

**Key idea:** The Taylor surrogate gives (1) closed-form leaf predictions and (2) a principled **split gain** used to pick the best (feature, threshold).

---

## How the split (feature + threshold) is chosen
At a node with aggregates \(G=\sum g_i,\;H=\sum h_i\), consider splitting into left/right children with \((G_L,H_L)\) and \((G_R,H_R)\). The **gain** (loss reduction) is
$$
\text{Gain}\;=\;\frac{1}{2}\!\left(\frac{G_L^2}{H_L+\lambda}+\frac{G_R^2}{H_R+\lambda}-\frac{G^2}{H+\lambda}\right)-\gamma.
$$
- Evaluate this **for each candidate threshold of each feature** (exact or histogram/quantile binned).
- Pick the split with **largest positive** Gain; if the best Gain \(\le 0\), **don’t split** (pruned by \(\gamma\)).

This is why modern GBDT libraries optimize **loss reduction via gradients/Hessians**, not Gini/entropy (those are for standalone classification trees/Random Forests).

---

## Symbol glossary (render-safe math)

**Feature vector**
$$x_i \in \mathbb{R}^d$$

**Target**
$$y_i \in \{0,1\}\ \text{or}\ \mathbb{R}$$

**Ensemble score after \(t\) rounds**
$$F^{(t)}(x) \in \mathbb{R}$$

**Prediction (e.g., logistic)**
$$\hat y_i^{(t)}=\sigma\!\big(F^{(t)}(x_i)\big),\quad \sigma(z)=\frac{1}{1+e^{-z}}$$

**Per-sample loss**
$$\ell(y_i,\hat y_i)$$

**Gradient (at current score \(F^{(t-1)}(x_i)\))**
$$g_i=\frac{\partial \ell(y_i,\hat y_i)}{\partial F}\Big|_{F=F^{(t-1)}(x_i)}$$

**Hessian (second derivative at current score)**
$$h_i=\frac{\partial^2 \ell(y_i,\hat y_i)}{\partial F^2}\Big|_{F=F^{(t-1)}(x_i)}$$

**New tree at round \(t\)**
$$f_t(x)$$

**Leaf index set**
$$I_j=\{\,i:\ x_i\ \text{falls into leaf } j\,\}$$

**Leaf value (score the tree outputs)**
$$w_j \in \mathbb{R}$$

**Aggregated gradient/Hessian in leaf \(j\)**
$$G_j=\sum_{i\in I_j} g_i,\qquad H_j=\sum_{i\in I_j} h_i$$

**Optimal leaf value (with L2)**
$$w_j^\*=-\frac{G_j}{H_j+\lambda}$$

**Regularization (XGBoost-style complexity)**
$$\Omega(f)=\gamma T+\frac{\lambda}{2}\sum_{j=1}^T w_j^2\quad\text{(optionally }+\ \alpha\,\lVert w\rVert_{1}\text{)}$$

**Split gain (parent \(G,H\) into left/right \(G_L,H_L\) and \(G_R,H_R\))**
$$
\text{Gain}
=
\frac{1}{2}\!\left(
\frac{G_L^2}{H_L+\lambda}
+
\frac{G_R^2}{H_R+\lambda}
-
\frac{G^2}{H+\lambda}
\right)-\gamma
$$

**Learning rate / ensemble update**
$$F^{(t)}(x)=F^{(t-1)}(x)+\eta\, f_t(x),\qquad \eta\in(0,1]$$

**Binary logistic specific derivatives**
$$g_i=\hat y_i^{(t-1)}-y_i,\qquad h_i=\hat y_i^{(t-1)}\big(1-\hat y_i^{(t-1)}\big)$$

---

## Logistic loss example (binary classification)
Let \(F\) be the score and \(\hat y=\sigma(F)=1/(1+e^{-F})\). With
$$
\ell(y,\hat y)=-\big(y\log \hat y+(1-y)\log(1-\hat y)\big),
$$
we have the derivatives
$$
g=\hat y-y,\qquad h=\hat y(1-\hat y).
$$
Plug these \(g,h\) into \(w^\*=-G/(H+\lambda)\) and the **Gain** formula above.
**Why keep \(h\)?** It yields a **Newton step in function space**, improving step size and split scoring vs. first-order residual fitting—especially important for logistic loss where gradients can saturate.

---

## Practical notes & gotchas
- **Not Gini/entropy:** Boosted trees pick splits by **loss reduction** from the Taylor surrogate, not impurity.
- **Thresholds:** exact (all unique values) vs. **histogram/quantile** binned (default in XGBoost/LightGBM/sklearn-hist).
- **Regularization interacts with splitting:** larger \(\lambda\), \(\gamma\), and `min_child_weight` (minimum child Hessian sum) suppress weak/noisy splits.
- **Missing values (NaN):** per split, learn a **default direction** (left/right) that maximizes Gain—no explicit imputation needed.
- **Early stopping + shrinkage:** low \(\eta\) with many trees and validation-based early stopping is a strong, reliable regularizer.

---

## Interview-style recap (one-liners)
- **Why Taylor expansion?** Quadratic surrogate \(\Rightarrow\) closed-form leaf values + principled split gains.
- **Why Hessians?** Curvature → Newton-like updates; faster, stabler convergence than residual-only fits.
- **How is the split chosen?** Evaluate Gain over (feature, threshold); take the **largest positive**; else stop splitting.


In [14]:
# Paste into a Jupyter *code* cell and run to render the notes as Markdown.
# (This version avoids LaTeX inside tables, which can break MathJax in some setups.)

from IPython.display import Markdown, display

md = r"""
# XGBoost vs. (Vanilla) GBDT — One-Cell Cheatsheet

## TL;DR
- **Both** are additive ensembles of CART trees trained stage-wise to minimize a loss.
- **Vanilla GBDT** typically uses **first-order** boosting (fit pseudo-residuals) with lighter explicit regularization.
- **XGBoost** uses a **second-order (Taylor)** approximation (gradients *and* Hessians), a **regularized objective**, and strong **systems optimizations**.

---

## Where the math differs (core formulas)

**Taylor (second-order) surrogate (per sample)**
\[
\ell(y, F+f) \approx \ell(y,F) + g\,f + \tfrac12 h\,f^2,
\quad
g=\frac{\partial \ell}{\partial F},\;
h=\frac{\partial^2 \ell}{\partial F^2}.
\]

**Leaf value in XGBoost (with L2 \(\lambda\))**
\[
w^\* \;=\; -\frac{G}{H+\lambda},
\quad
G=\sum g_i,\; H=\sum h_i \text{ over the leaf.}
\]

**Split gain (parent \(G,H\) → children \(G_L,H_L\), \(G_R,H_R\))**
\[
\text{Gain}
=
\frac{1}{2}\!\left(
\frac{G_L^2}{H_L+\lambda}
+
\frac{G_R^2}{H_R+\lambda}
-
\frac{G^2}{H+\lambda}
\right) - \gamma .
\]

**Binary logistic derivatives (score \(F\), \(\hat y=\sigma(F)\))**
\[
g=\hat y-y,
\qquad
h=\hat y(1-\hat y),
\qquad
\hat y=\frac{1}{1+e^{-F}}.
\]

---

## Conceptual comparison (bullets instead of a table)

**Update type**
- *GBDT*: First-order (fit residuals) or gradient-only.
- *XGBoost*: Second-order (uses \(g\) and \(h\)) → Newton-like step.

**Objective**
- *GBDT*: Empirical loss (regularization mostly via depth, min samples, shrinkage).
- *XGBoost*: Loss **+** complexity
  \[
  \Omega(f)=\gamma T+\frac{\lambda}{2}\sum_j w_j^2
  \quad (\text{optionally } + \alpha \|w\|_1).
  \]

**Split criterion**
- *GBDT*: Loss reduction (often residual SSE for regression).
- *XGBoost*: **Regularized Gain** (formula above), pruned if \(\le 0\).

**Missing values**
- *GBDT*: Usually impute/ignore.
- *XGBoost*: Learns a default direction (left/right) per split to maximize Gain.

**Performance**
- *GBDT*: Library-dependent; exact or histogram splits.
- *XGBoost*: Histogram/quantile sketch, column/row subsampling, parallelism, out-of-core.

**Regularization knobs**
- *GBDT*: `learning_rate`, `max_depth`, `min_samples_leaf`, early stopping.
- *XGBoost*: Above **plus** `reg_lambda` (λ), `reg_alpha` (α), `gamma` (γ), `min_child_weight` (min Hessian mass).

---

## Practical implications

- **Stability & convergence**: Second-order updates give better step sizes and split scoring (especially for logistic loss).
- **Overfitting control**: Shrinkage + early stopping in both; XGBoost’s λ/α/γ/min_child_weight directly appear in the split math.
- **Sparsity/missingness**: XGBoost’s learned default direction often beats naïve imputation.
- **Speed/scale**: XGBoost’s hist + subsampling + threading is very fast on large/sparse data.

---

## When to use which (rules of thumb)

- Choose **XGBoost** for large/sparse/imbalanced data, logistic loss, or when you want stronger defaults and native missing handling.
- A **simple GBDT** can be fine for small/moderate data and quick prototypes.

---

## Minimal set to memorize

\[
w^\* = -\frac{G}{H+\lambda}
\qquad
\text{Gain} = \tfrac12\!\left(\frac{G_L^2}{H_L+\lambda} + \frac{G_R^2}{H_R+\lambda} - \frac{G^2}{H+\lambda}\right) - \gamma
\qquad
g=\hat y-y,\; h=\hat y(1-\hat y)
\]

"""

display(Markdown(md))



# XGBoost vs. (Vanilla) GBDT — One-Cell Cheatsheet

## TL;DR
- **Both** are additive ensembles of CART trees trained stage-wise to minimize a loss.
- **Vanilla GBDT** typically uses **first-order** boosting (fit pseudo-residuals) with lighter explicit regularization.
- **XGBoost** uses a **second-order (Taylor)** approximation (gradients *and* Hessians), a **regularized objective**, and strong **systems optimizations**.

---

## Where the math differs (core formulas)

**Taylor (second-order) surrogate (per sample)**
\[
\ell(y, F+f) \approx \ell(y,F) + g\,f + \tfrac12 h\,f^2,
\quad
g=\frac{\partial \ell}{\partial F},\;
h=\frac{\partial^2 \ell}{\partial F^2}.
\]

**Leaf value in XGBoost (with L2 \(\lambda\))**
\[
w^\* \;=\; -\frac{G}{H+\lambda},
\quad
G=\sum g_i,\; H=\sum h_i \text{ over the leaf.}
\]

**Split gain (parent \(G,H\) → children \(G_L,H_L\), \(G_R,H_R\))**
\[
\text{Gain}
=
\frac{1}{2}\!\left(
\frac{G_L^2}{H_L+\lambda}
+
\frac{G_R^2}{H_R+\lambda}
-
\frac{G^2}{H+\lambda}
\right) - \gamma .
\]

**Binary logistic derivatives (score \(F\), \(\hat y=\sigma(F)\))**
\[
g=\hat y-y,
\qquad
h=\hat y(1-\hat y),
\qquad
\hat y=\frac{1}{1+e^{-F}}.
\]

---

## Conceptual comparison (bullets instead of a table)

**Update type**
- *GBDT*: First-order (fit residuals) or gradient-only.
- *XGBoost*: Second-order (uses \(g\) and \(h\)) → Newton-like step.

**Objective**
- *GBDT*: Empirical loss (regularization mostly via depth, min samples, shrinkage).
- *XGBoost*: Loss **+** complexity
  \[
  \Omega(f)=\gamma T+\frac{\lambda}{2}\sum_j w_j^2
  \quad (\text{optionally } + \alpha \|w\|_1).
  \]

**Split criterion**
- *GBDT*: Loss reduction (often residual SSE for regression).
- *XGBoost*: **Regularized Gain** (formula above), pruned if \(\le 0\).

**Missing values**
- *GBDT*: Usually impute/ignore.
- *XGBoost*: Learns a default direction (left/right) per split to maximize Gain.

**Performance**
- *GBDT*: Library-dependent; exact or histogram splits.
- *XGBoost*: Histogram/quantile sketch, column/row subsampling, parallelism, out-of-core.

**Regularization knobs**
- *GBDT*: `learning_rate`, `max_depth`, `min_samples_leaf`, early stopping.
- *XGBoost*: Above **plus** `reg_lambda` (λ), `reg_alpha` (α), `gamma` (γ), `min_child_weight` (min Hessian mass).

---

## Practical implications

- **Stability & convergence**: Second-order updates give better step sizes and split scoring (especially for logistic loss).
- **Overfitting control**: Shrinkage + early stopping in both; XGBoost’s λ/α/γ/min_child_weight directly appear in the split math.
- **Sparsity/missingness**: XGBoost’s learned default direction often beats naïve imputation.
- **Speed/scale**: XGBoost’s hist + subsampling + threading is very fast on large/sparse data.

---

## When to use which (rules of thumb)

- Choose **XGBoost** for large/sparse/imbalanced data, logistic loss, or when you want stronger defaults and native missing handling.
- A **simple GBDT** can be fine for small/moderate data and quick prototypes.

---

## Minimal set to memorize

\[
w^\* = -\frac{G}{H+\lambda}
\qquad
\text{Gain} = \tfrac12\!\left(\frac{G_L^2}{H_L+\lambda} + \frac{G_R^2}{H_R+\lambda} - \frac{G^2}{H+\lambda}\right) - \gamma
\qquad
g=\hat y-y,\; h=\hat y(1-\hat y)
\]



# 🌲 Random Forest (RF) Notes

## What is Random Forest?
- Ensemble method built from **many decision trees**.
- Uses **bagging (bootstrap aggregation)** + **random feature selection**.
- Final prediction = **majority vote (classification)** or **average (regression)**.

---

## Key Ideas
1. **Bootstrap Sampling**: each tree trained on random sample of data (with replacement).
2. **Random Feature Selection**: only a random subset of features considered at each split → decorrelates trees.
3. **Aggregation**: combine predictions from all trees.

---

## Why it Works
- Decision trees = high variance learners.
- Bagging reduces variance by averaging many models.
- Random features ensure diversity → less correlated errors.

---

## Strengths
- Robust against overfitting compared to single trees.
- Handles high-dimensional data well.
- Provides **feature importance** estimates.

## Weaknesses
- Less interpretable than a single tree.
- Can be slower with many trees.
- May still struggle on highly imbalanced datasets.

---

## Typical Hyperparameters
- **n_estimators**: number of trees (larger = less variance).
- **max_features**: features considered at each split.
- **max_depth**: max tree depth (controls overfitting).
- **min_samples_split / min_samples_leaf**: smoothness of predictions.

---

## Random Forest vs GBDT
- **RF**: parallel trees, bagging, variance reduction.
- **GBDT**: sequential trees, boosting, bias reduction.


# 📒 XGBoost Split Math & Why It’s Fast (gᵢ, hᵢ, f*, Gain)

---

## 1) Setup: Boosting with a New Tree
At boosting step \(t\), we add a tree \(f\) to current predictions \(\hat{y}^{(t-1)}\):

$$
\text{Obj}^{(t)} = \sum_i l\big(y_i, \hat{y}_i^{(t-1)} + f(x_i)\big) + \Omega(f)
$$

We do a **2nd-order Taylor expansion** around \(\hat{y}_i^{(t-1)}\):

$$
l\big(y_i, \hat{y}_i^{(t-1)} + f(x_i)\big) \approx l(y_i,\hat{y}_i) + g_i f(x_i) + \tfrac{1}{2} h_i f(x_i)^2
$$

where

$$
g_i=\frac{\partial l}{\partial \hat{y}}\Big|_{\hat{y}=\hat{y}_i^{(t-1)}},\quad
h_i=\frac{\partial^2 l}{\partial \hat{y}^2}\Big|_{\hat{y}=\hat{y}_i^{(t-1)}}
$$

---

## 2) Leaf Value Is a Scalar \(f\) (Per Leaf)
Each leaf outputs a constant \(f\). All samples in the leaf get their prediction shifted by \(f\).

Define leaf sums:

$$
G=\sum_{i\in \text{leaf}} g_i,\quad H=\sum_{i\in \text{leaf}} h_i
$$

The approximated objective for one leaf with L2 penalty \(\lambda\):

$$
\text{Obj}_\text{leaf} = G f + \tfrac12 (H+\lambda) f^2
$$

---

## 3) Why the Optimal \(f^*\) Depends on G, H, λ
Take derivative wrt \(f\):

$$
\frac{\partial \text{Obj}_\text{leaf}}{\partial f} = G + (H+\lambda) f
$$

Set to zero:

$$
G + (H+\lambda) f^* = 0
$$

Solve:

$$
f^* = -\frac{G}{H+\lambda}
$$

**Interpretation**
- \(G = \sum g_i\): how strong the gradients push the predictions in one direction.
- \(H = \sum h_i\): curvature (confidence in that direction).
- \(\lambda\): regularization, prevents extreme weights when \(H\) is small.

So \(f^*\) is essentially a **Newton step**: gradient / curvature, with regularization for stability.

---

## 4) Minimum Leaf Contribution
Plugging back into the objective:

$$
\text{Obj}_\text{leaf}^{\min} = -\tfrac12 \frac{G^2}{H+\lambda}
$$

This value is used to compare different split structures.

---

## 5) Split Gain Formula (Parent → Left/Right)
For a candidate split that partitions samples into **Left (L)** and **Right (R)**:

- Parent:
  $$ G=G_L+G_R,\; H=H_L+H_R $$

- Gain from split:

$$
\text{Gain}=\tfrac12\Big(\frac{G_L^2}{H_L+\lambda}+\frac{G_R^2}{H_R+\lambda}-\frac{G^2}{H+\lambda}\Big)-\gamma
$$

- \(\gamma\): penalty for adding a leaf (controls complexity).

---

## 6) Why This Is Fast
- Compute \(g_i,h_i\) **once per sample** per boosting round.
- Split search uses only **aggregated sums** (\(G,H\)) via prefix sums.
- Closed-form \(f^*\) means **no search over leaf outputs**.
- Each split evaluation reduces to simple arithmetic → much faster than recomputing the full loss.

---

## 7) TL;DR
- \(g_i,h_i\) = per-sample gradient & curvature of the loss.
- Optimal leaf weight:
  $$ f^* = -\frac{G}{H+\lambda} $$
- This formula comes from minimizing the quadratic approximation, making training efficient.
- Split gain is then derived entirely from these aggregated terms, enabling fast search across thresholds.
